In [13]:
import tensorflow as tf
import pandas as pd
import numpy as np
import json
import os
from utils.char_tokenizer import CharTokenizer

In [14]:
config = json.load(open("config.json"))

In [15]:
data = pd.read_csv(os.path.join(config["data"]["path"], "data.csv"))
data = data.drop({"Unnamed: 0"}, axis=1)
data['product_type'] = data['product_type.value']
data = data[['description', 'description2', 'path', 'product_type', 'label']]
data

,description,description2,path,product_type,label
0,whole foods market body scrub & body butter du...,cellphonecoverb019ol9e02amazon brand - solimo ...,66/66f3a68c.jpg,CELLULAR_PHONE_CASE,0
1,mobile cover,b07pqbdbc4starter performance slide sandalia p...,5d/5d4519b7.jpg,CELLULAR_PHONE_CASE,0
2,mobile cover,b07rgqgyvzamazon brand – stone & beam modern r...,d9/d9f55153.jpg,PLANTER,0
3,b082dft9hpnubeno men's brown formal shoes-9 uk...,formal shoes for mens leather,cc/cc2759da.jpg,SHOES,1
4,amazon brand - solimo designer head phone 3d p...,b07trw4xwzamazon brand - solimo designer head ...,49/49b1b22b.jpg,CELLULAR_PHONE_CASE,1
...,...,...,...,...,...
652299,b01n365np7amazon brand - solimo self-adhering ...,amazonbasics ratchet strap - pack of 4; td0901...,c5/c597deaa.jpg,CELLULAR_PHONE_CASE,0
652300,gifts heramazonbasics stainless steel kettle w...,mobile coverb07vtbjtdc,17/1714da56.jpg,CELLULAR_PHONE_CASE,0
652301,amazon brand - solimo designer sparkle girl bo...,back coverb0854l5ndjamazon brand - solimo desi...,36/368ecbc0.jpg,CELLULAR_PHONE_CASE,1
652302,ratchingb088w5dhsqamazonbasics 45-degree offse...,ratchingb088w5dhsqamazonbasics 45-degree offse...,b2/b2bbf71b.jpg,TOOLS,1


In [16]:
img = data[["path"]] # two brackets for keeping the column name
text = data["description"]
text2 = data["description2"]
product_types = data[["product_type"]]
labels = data[["label"]]

In [17]:
tk = CharTokenizer(config["data"]["alphabet"])
text = tk.tokenize(text)
text.shape

(652304, 1014)

In [18]:
weights = tk.create_embedding_weights()
weights.shape

(3443, 3442)

In [19]:
from models.char_cnn_zhang import CharCNNZhang

In [20]:
model = CharCNNZhang(input_size=config["data"]["input_size"],
                    embedding_size=len(tk.word_index),
                    conv_layers=config["char_cnn_zhang"]["conv_layers"],
                    fully_connected_layers=config["char_cnn_zhang"]["fully_connected_layers"],
                    output_size=config["char_cnn_zhang"]["output_size"],
                    embedding_weights=weights)

2023-05-22 12:30:47.180202: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-05-22 12:30:47.180290: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kmazhar-cpu2): /proc/driver/nvidia/version does not exist
2023-05-22 12:30:47.182456: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


CharCNNZhang model built: 
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sent_input (InputLayer)     [(None, 1014)]            0         
                                                                 
 embedding (Embedding)       (None, 1014, 3442)        11850806  
                                                                 
 conv1d (Conv1D)             (None, 1008, 256)         6168320   
                                                                 
 activation (Activation)     (None, 1008, 256)         0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 336, 256)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 330, 256)          459008    
                                  